**This jupyter notebook file is for testing out the code to generate travel plan to different places in different country countries**

In [1]:
# Set up key from OpenAI API

#Generate separate python file call sk that contain my OpenAI API Key called my_sk
from sk import my_sk
import os
os.environ['OPENAI_API_KEY'] = my_sk

In [7]:
#import neccessary libraries
#from langchain_community.llms import OpenAI 
from langchain_openai import OpenAI #You can use other opensource model from huggingface
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [5]:
#pip install -U langchain-openai

In [3]:
#create llm model
llm = OpenAI(temperature = 0.6)
#testing the model
response = llm("I want to travel to Japan. Suggest some best places to go visit.")
print(response)

/Users/siriluk/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. Tokyo: The bustling capital city of Japan, Tokyo is a must-visit for any traveler. It offers a unique blend of traditional culture and modern technology, with attractions such as the Tokyo Skytree, Meiji Shrine, and the bustling Shibuya district.

2. Kyoto: Known for its numerous temples, shrines, and traditional gardens, Kyoto is a must-visit for those interested in Japanese history and culture. It is also home to the famous Kinkaku-ji (Golden Pavilion) and Fushimi Inari Shrine.

3. Osaka: The food capital of Japan, Osaka is a must-visit for foodies. It is also home to attractions such as Osaka Castle, Universal Studios Japan, and the vibrant Dotonbori district.

4. Nara: Known for its friendly deer and ancient temples, Nara is a charming city that offers a peaceful escape from the bustling cities of Tokyo and Osaka.

5. Hiroshima: This city is known for its tragic history as the site of the atomic bombing during World War II. It is now a peaceful and modern city, with attraction

In [4]:
response = llm("I want to travel to Vietnam. Suggest 5 most popular places to go visit. Do not give an explanation of each place")
print(response)



1. Ha Long Bay
2. Hoi An
3. Ho Chi Minh City
4. Nha Trang
5. Da Nang


## Prompt template

In [5]:
# generate a prompt template
prompt_name = PromptTemplate(
    name = "places_to_visit",
    input_variables = ['country'],
    template = "I want to travel to {country}. Suggest 5 most popular places to go visit. Do not give an explanation of each place"
)
prompt_name.format(country = "Vietnam")

'I want to travel to Vietnam. Suggest 5 most popular places to go visit. Do not give an explanation of each place'

## Chain

In [6]:
# Now using chain from langchain
chain = LLMChain(llm = llm, prompt=prompt_name)
places = chain.run("Thailand") #run to get suggestion or answer
print(places)

/Users/siriluk/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. Bangkok
2. Phuket
3. Chiang Mai
4. Krabi
5. Koh Samui


## SequentialChain

In [30]:
# First prompt
llm = OpenAI(temperature = 0.6)
prompt_temp_places = PromptTemplate(
    input_variables=['country'],
    template="I want to travel to {country}. Suggest 5 most popular places to go visit. Do not give an explanation of each place"
)

places_chain = LLMChain(llm=llm, prompt=prompt_temp_places, 
                        output_key="places_name")
print(places_chain.run('Switzerland'))



1. Zurich
2. Geneva
3. Lucerne
4. Interlaken
5. Zermatt


In [36]:
# Second prompt
promp_temp_spots = PromptTemplate(
    input_variables=['places_name'],
    template="Suggest popular spots and popular restaurants for each {places_name}"
)

spots_food_chain = LLMChain(llm=llm, prompt=promp_temp_spots,
                                  output_key="spot_food_places")

print(spots_food_chain.run('Zurich'))

 district

District 1 (Altstadt): 
1. Grossmünster Church - a historic landmark and one of the most famous churches in Zurich
2. Bahnhofstrasse - a popular shopping street with luxury stores and cafes
3. Fraumünster Church - known for its stunning stained glass windows by artist Marc Chagall
4. Restaurant Zeughauskeller - a traditional Swiss restaurant with a cozy atmosphere and hearty dishes
5. Lindenhof Hill - a scenic spot with views of the city and the Limmat River

District 2 (Enge): 
1. Lake Zurich - a popular spot for swimming, boating, and picnicking
2. Rote Fabrik - a cultural center with concerts, exhibitions, and a lakeside restaurant
3. Restaurant Seerose - a lakeside restaurant with a beautiful terrace and delicious Swiss cuisine
4. My Kitchen - a trendy restaurant serving international fusion cuisine
5. Enge Church - a beautiful Neo-Gothic church with a distinctive red brick facade

District 3 (Wiedikon): 
1. Langstrasse - a lively street with bars, restaurants, and stree

In [37]:
# third prompt
promp_temp_plan = PromptTemplate(
    input_variables=['spot_food_places'],
    template='''
        create a plan for one week to visit popular spots and popular restaurants in {spot_food_places}. 
        '''
)
plan_chain = LLMChain(llm=llm, prompt=promp_temp_plan,
                                  output_key="one_week_plan")

print(plan_chain.run('Zurich'))


Day 1:
- Arrive in Zurich and check into hotel
- Take a walk around the Old Town to see historic landmarks such as Grossmünster Church and Fraumünster Church
- Have lunch at Zeughauskeller, a popular Swiss restaurant known for its traditional cuisine
- Spend the afternoon at Lake Zurich, taking a boat ride and enjoying the views
- In the evening, head to Langstrasse for a lively nightlife scene and try some local beers at a bar

Day 2:
- Start the day with a visit to the Swiss National Museum to learn about the country's history and culture
- Have lunch at Hiltl, the world's oldest vegetarian restaurant
- Spend the afternoon at Zurich Zoo, home to over 380 different species
- In the evening, head to the trendy district of Kreis 5 for dinner at Frau Gerolds Garten, a popular outdoor restaurant with a unique atmosphere

Day 3:
- Take a day trip to Mount Rigi, known as the "Queen of the Mountains" for its stunning views of the Swiss Alps
- Have lunch at one of the restaurants at Rigi Kul

## Put the above in SequenctialChains

In [43]:
chain = SequentialChain(
    chains=[places_chain, spots_food_chain, plan_chain],
    input_variables=['country'],
    output_variables=['places_name', 'spot_food_places', 'one_week_plan']
)


chain({'country':'Thailand'})

{'country': 'Thailand',
 'places_name': '\n\n1. Bangkok\n2. Phuket\n3. Chiang Mai\n4. Krabi\n5. Koh Samui',
 'spot_food_places': '\n\n1. Bangkok:\n- Grand Palace\n- Wat Phra Kaew (Temple of the Emerald Buddha)\n- Chatuchak Weekend Market\n- Khao San Road\n- Wat Arun (Temple of Dawn)\n- Chinatown\n- Siam Paragon Shopping Mall\n- Lumpini Park\n- MBK Center\n- Asiatique The Riverfront\n\nPopular restaurants:\n- Gaggan (Innovative Indian cuisine)\n- Nahm (Michelin-starred Thai cuisine)\n- Jay Fai (Street food, especially crab omelette)\n- Issaya Siamese Club (Upscale Thai fusion)\n- Somtum Der (Authentic Isaan cuisine)\n- Supanniga Eating Room (Traditional Thai dishes with a modern twist)\n- Thip Samai (Pad Thai)\n- Baan Khanitha (Fine dining Thai cuisine)\n- Blue Elephant (Royal Thai cuisine)\n- Bo.Lan (Sustainable farm-to-table Thai dishes)\n\n2. Phuket:\n- Patong Beach\n- Phang Nga Bay\n- Big Buddha Phuket\n- Kata Beach\n- Phi Phi Islands\n- Old Phuket Town\n- Bangla Road (nightlife)\n-